1)	For Each Procedure and each main category report on the trend changes.
    a)	Find the difference between Past (6/3/2/1) month average and Current months value 
    for Cost Per unit, Total Units per procedure, Total revenue etc. and this will be the slope
    If the slope is positive then flag it , else if negative then it has improved else if it’s zero then it has maintained         same.


In [77]:
import pandas as pd
import numpy as np
import datetime
import os

In [78]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/IHS CPP Analysis")

In [6]:
# reading the data
data = pd.read_excel('Complete_data_after_join.xlsx')
# getting an overview of our data
print("Our data has {0} rows and {1} columns".format(data.shape[0], data.shape[1]))
# checking for missing values
print("Are there missing values? {}".format(data.isnull().any().any()))

Our data has 147763 rows and 15 columns
Are there missing values? False


In [7]:
data['Month']= pd.DatetimeIndex(data['com_procedure_date']).month

In [11]:
data['Year']= pd.DatetimeIndex(data['com_procedure_date']).year

In [16]:
only_2019=data[data['Year']==2019]

In [19]:
data=only_2019

In [20]:
print(" month1 is the first month of the past months \n month2 is the last month of the past months \n month3 is the current month (NOTE: month1 always less than month2)")


 month1 is the first month of the past months 
 month2 is the last month of the past months 
 month3 is the current month (NOTE: month1 always less than month2)


In [21]:
month1 = int(input(prompt='Please enter month 1 : '))

Please enter month 1 : 5


In [22]:
month2 = int(input(prompt='Please enter month 2 : '))

Please enter month 2 : 7


In [23]:
month3 = int(input(prompt='Please enter month 3 : '))

Please enter month 3 : 8


In [24]:
pivot_table_cpu=pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category'],columns='Month', values=["com_cost_price",'com_quantity_consumed'],aggfunc=np.sum).apply(np.ceil)

In [25]:
flat_table_cpu=pd.DataFrame(pivot_table_cpu.to_records())

In [79]:
flat_table_cpu.to_excel('pivot_table.xlsx')

In [26]:
# take input from user on current month and use that below to remove it from the dataset
# This line helps in removing those records from the table whose current month value is null
ip=str(('com_cost_price', month3))
qp=str(('com_quantity_consumed', month3))
flat_table_cpu.dropna(subset=[qp], inplace=True)
flat_table_cpu.dropna(subset=[ip], inplace=True)

In [27]:
flat_table_cpu.fillna(value=0.0,inplace=True)

In [28]:
colname=data["Month"].unique()
colname=colname.tolist()
colname=[str(i) for i in colname]
colname.insert(0,'Main_category')
colname.insert(0,'Procedure_Name')

In [30]:
def avg_cpu_table(flat_table_cpu,colname):
    Cols =colname
    Result=pd.DataFrame(columns=Cols)
    for procedure in (flat_table_cpu['com_procedurename'].unique()).tolist():
        main_category = (flat_table_cpu.loc[flat_table_cpu['com_procedurename']== procedure]['com_bom_main_category']).tolist()
        for category in main_category:
            Main_dict={}
            row_data=flat_table_cpu[(flat_table_cpu['com_procedurename']== procedure)&(flat_table_cpu['com_bom_main_category']== category)]
            for month in range(1,month3+1):
                tot_qty=row_data[str(('com_quantity_consumed', month))].tolist()[0]
                tot_cost=row_data[str(('com_cost_price', month))].tolist()[0]
                sub_dict={'Total cost price':tot_cost,'Total qty consumed':tot_qty}
                Main_dict.update({str(month):sub_dict})
                Main_dict.update({'Procedure_Name':procedure,'Main_category':category})
            Result= Result.append(Main_dict,ignore_index=True)
    return Result
                
                
                
                
    
    

In [31]:
op=avg_cpu_table(flat_table_cpu,colname)

In [32]:
# This function helps in finding the trend change or slope. 
#Idea is to get the avg of Cost per unit for past required num of months and compare the avg with current month's avg.
#Slope or trend calculation: we take difference b/w past months avg and that of current month.
# If the difference is positve then Cost per unit has increased, if negative means it has decreased which is good and if 0 then it is constant 
#Here we take three i/p's 
# month1 is the first month of the past months
# month2 is the last month of the past months
# month3 is the current month (NOTE: month1 always less than month2)
# CONSIDER SUM AND NOT AVERAGE

def trend_change_avg_cpu(month1, month2, month3, flat_table_cpu):
    flat_table=flat_table_cpu
    Cols = ['Procedure_Name','Main_category','%Change_in_average_cost_price']
    Result=pd.DataFrame(columns=Cols)
    for procedure in (flat_table['Procedure_Name'].unique()).tolist():
        main_category = (flat_table.loc[flat_table['Procedure_Name']== procedure]['Main_category']).tolist()
        for category in main_category:
            row_data=(flat_table.loc[(flat_table['Procedure_Name']==procedure)&(flat_table['Main_category']==category)])
            row_data.set_index(['Procedure_Name', 'Main_category'],inplace=True)
            past=row_data.loc[:, str(month1):str(month2)]
            current=row_data[str(month3)][procedure][category]
            total_cost_price_sum=0.0
            total_qty=0.0
            for i in range(month1,month2+1):
                if (past[str(i)][procedure][category]) is None:
                    continue
                else:
                    total_cost_price_sum +=past[str(i)][procedure][category].get('Total cost price')
                    total_qty +=past[str(i)][procedure][category].get('Total qty consumed')
            
            if (total_cost_price_sum==0.0):
                past_avg_cost_price=0.0
            else:
                past_avg_cost_price=(total_cost_price_sum/total_qty)
            
            if current is None:
                curr_avg_cost_price=0.0
            else:
                curr_avg_cost_price=(current.get('Total cost price')/current.get('Total qty consumed'))
                
            if(past_avg_cost_price>0.0):
                change=((curr_avg_cost_price-past_avg_cost_price)/past_avg_cost_price)*100
            else:
                change=((curr_avg_cost_price-past_avg_cost_price)/curr_avg_cost_price)*100

                
            Result= Result.append({'Procedure_Name':procedure,'Main_category':category,'%Change_in_average_cost_price':change},ignore_index=True)
    
    Result.sort_values(by='%Change_in_average_cost_price', ascending=False,inplace=True)
    return Result
    

In [33]:
result= trend_change_avg_cpu(month1,month2,month3,op)

In [38]:
result.to_excel('Avg_cost_per_unit_result '+str(month1)+' - '+str(month2)+'.xlsx')

# Average Quantity per procedure

In [39]:
pivot_table_AQPP=pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category'],columns='Month', values=["com_quantity_consumed","com_procedure_number"],aggfunc={"com_quantity_consumed":np.sum,"com_procedure_number":pd.Series.nunique}).apply(np.ceil)

In [40]:
flat_table_AQPP=pd.DataFrame(pivot_table_AQPP.to_records())

In [41]:
# take input from user on current month and use that below to remove it from the dataset
# This line helps in removing those records from the table whose current month value is null
ip=str(('com_procedure_number',month3))
qp=str(('com_quantity_consumed', month3))
flat_table_AQPP.dropna(subset=[qp], inplace=True)
flat_table_AQPP.dropna(subset=[ip], inplace=True)

In [42]:
flat_table_AQPP.fillna(value=0.0,inplace=True)

In [43]:
#Below function generates a table where we have by procedure, by main category avg qty per procedure for each month. 
#This table can be used to get a hplistic picture and produce graphs and charts which is similar to what is already there in dashboard
#I/p is a pivot table which has total qty and number of procedures monthwise
def avg_qty_pp_table(flat_table_AQPP, colname):
    Result=pd.DataFrame(columns=colname)
    for procedure in (flat_table_AQPP['com_procedurename'].unique()).tolist():
        main_category = (flat_table_AQPP.loc[flat_table_AQPP['com_procedurename']== procedure]['com_bom_main_category']).tolist()
        for category in main_category:
            Main_dict={}
            row_data=flat_table_AQPP[(flat_table_AQPP['com_procedurename']==procedure)&(flat_table_AQPP['com_bom_main_category']== category)]
            for month in range(1,month3+1):
                tot_qty=row_data[str(('com_quantity_consumed', month))].tolist()[0]
                tot_proc=row_data[str(('com_procedure_number', month))].tolist()[0]
                sub_dict={'Total Proc':tot_proc,'Total qty consumed':tot_qty}
                Main_dict.update({str(month):sub_dict})
            Main_dict.update({'Procedure_Name':procedure,'Main_category':category})
            Result= Result.append(Main_dict,ignore_index=True)
    return Result
    

In [44]:
check= avg_qty_pp_table(flat_table_AQPP,colname)

In [49]:
# This function helps in finding the trend change or slope. 
#Idea is to get the avg of qty per procedure for past required num of months and compare the avg with current month's avg.
#Slope or trend calculation: we take difference b/w past months avg and that of current month.
# If the difference is positve then qty per procedure has increased, if negative means it has decreased which is good and if 0 then it is constant 
#Here we take three i/p's 
# month1 is the first month of the past months
# month2 is the last month of the past months
# month3 is the current month (NOTE: month1 always less than month2)
def trend_change_avg_qty_pp(month1, month2, month3,data_table):
    Cols = ['Procedure_Name','Main_category','%Change_in_average_qty_pp']
    Result=pd.DataFrame(columns=Cols)
    for procedure in (data_table['Procedure_Name'].unique()).tolist():
        main_category = (data_table.loc[data_table['Procedure_Name']== procedure]['Main_category']).tolist()
        for category in main_category:
            row_data=(data_table.loc[(data_table['Procedure_Name']== procedure) & (data_table['Main_category']==category)])
            row_data.set_index(['Procedure_Name', 'Main_category'],inplace=True)
            past=row_data.loc[:, str(month1):str(month2)]
            current=row_data[str(month3)][procedure][category]
            total_proc=0.0
            total_qty=0.0
            for i in range(month1,month2+1):
                if (past[str(i)][procedure][category]) is None:
                    continue
                else:
                    total_proc +=past[str(i)][procedure][category].get('Total Proc')
                    total_qty +=past[str(i)][procedure][category].get('Total qty consumed')
            
            if (total_proc==0.0):
                past_avg_qty_pp=0.0
            else:
                past_avg_qty_pp=(total_qty/total_proc)*100
            
            if current is None:
                curr_avg_qty_pp=0.0
            else:
                curr_avg_qty_pp=(current.get('Total qty consumed')/current.get('Total Proc'))*100
                
            if(past_avg_qty_pp>0.0):
                change=((curr_avg_qty_pp-past_avg_qty_pp)/past_avg_qty_pp)*100
            else:
                change=((curr_avg_qty_pp-past_avg_qty_pp)/curr_avg_qty_pp)*100
            
            Result= Result.append({'Procedure_Name':procedure,'Main_category':category,'%Change_in_average_qty_pp':change},ignore_index=True)
    
    Result.sort_values(by='%Change_in_average_qty_pp', ascending=False,inplace=True)
    return Result


In [50]:
output= trend_change_avg_qty_pp(month1, month2, month3,check)

In [51]:
output.to_excel('Avg_qty_pp_result '+str(month1)+' - '+str(month2)+'.xlsx')

2) At Supplier or Manufacturer level report on the growth in terms of total revenue. It can also be broken By procedure, By category which supplier has grown more.

# Supplier level Analysis

In [52]:
Month_wise_total_expen=pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category','Month'],values=["com_cost_price"],aggfunc={'com_cost_price': np.sum}).apply(np.ceil)

In [53]:
Month_wise_total_expen = pd.DataFrame(Month_wise_total_expen.to_records())

In [54]:
pt_tot_expend = pd.pivot_table(data, index = ['com_procedurename','com_bom_main_category','Month'],columns='com_supplier', values=["com_cost_price","com_quantity_consumed"],aggfunc={'com_cost_price': np.mean,'com_quantity_consumed':np.sum})

In [55]:
def cheapest_supplier_table(pt_tot_expend,Month_wise_total_expen,colname):
    flat_table=pd.DataFrame(pt_tot_expend.to_records())
    Result = pd.DataFrame(columns=colname)
    for procedure in (flat_table['com_procedurename'].unique()).tolist():
        main_category = (flat_table.loc[flat_table['com_procedurename']== procedure]['com_bom_main_category'].unique()).tolist()
        for category in main_category:
            Main_dict={}
            months=(flat_table.loc[(flat_table['com_procedurename']== procedure)&(flat_table['com_bom_main_category']==category)]['Month']).tolist()
            for month in months:
                extracted_cost_price=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category) &(Month == @month)').dropna(axis=1).loc[:,('com_cost_price',slice(None))]
                extracted_qty=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category) &(Month == @month)').dropna(axis=1).loc[:,'com_quantity_consumed']
                Cheapest_supplier_name=list(extracted_cost_price.idxmin(axis=1).values[0])[1]
                Cheapest_supp_qty=extracted_qty[Cheapest_supplier_name].values[0]
                Cheapest_cost_price=list(extracted_cost_price.min(axis=1))[0]
                Cheapest_supp_expend=Cheapest_cost_price*Cheapest_supp_qty
                Total_expend=Month_wise_total_expen[(Month_wise_total_expen['com_procedurename']==procedure)&(Month_wise_total_expen['com_bom_main_category']==category)&(Month_wise_total_expen['Month']==month)]['com_cost_price'].tolist()[0]
                sub_dict={'Supplier_Name':Cheapest_supplier_name,'Qty':Cheapest_supp_qty,'Cost_Price':Cheapest_cost_price,'Cheapest_supp_expend':Cheapest_supp_expend,'Total Expend': Total_expend}
                Main_dict.update({str(month):sub_dict})
            Main_dict.update({'Procedure_Name':procedure,'Main_category':category})
            Result=Result.append(Main_dict,ignore_index=True)
        
    return Result        
        

In [56]:
Cheap_supp_info=cheapest_supplier_table(pt_tot_expend, Month_wise_total_expen,colname)

In [57]:
def cheapest_supplier_trend(table_of_cheap_supp,month1,month2,month3):
    Result = pd.DataFrame(columns=['Procedure','Main_Category','Is_curr_%_of_CS_expend_more'])
    for procedure in (table_of_cheap_supp['Procedure_Name'].unique()).tolist():
        main_category = (table_of_cheap_supp.loc[table_of_cheap_supp['Procedure_Name']== procedure]['Main_category']).tolist()
        for category in main_category:
            row_data=(table_of_cheap_supp.loc[(table_of_cheap_supp['Procedure_Name']==procedure)&(table_of_cheap_supp['Main_category']==category)])
            row_data.set_index(['Procedure_Name', 'Main_category'],inplace=True)
            past=row_data.loc[:, str(month1):str(month2)]
            past.fillna(0, inplace=True)
            current=row_data.loc[:, str(month3):str(month3)]
            current.fillna(0, inplace=True)
            total_cheap_sup_exp=0.0
            tot_exp=0.0
            for i in range(month1,month2+1):
                if ((past[str(i)][procedure][category])==0):
                    continue
                else:
                    total_cheap_sup_exp +=past[str(i)][procedure][category].get('Cheapest_supp_expend')
                    tot_exp +=past[str(i)][procedure][category].get('Total Expend')
            
            if (tot_exp==0.0):
                past_cheap_supp_avg_amt=0.0
            else:
                past_cheap_supp_avg_amt=(total_cheap_sup_exp/tot_exp)*100
            
            if ((current[str(month3)][procedure][category]) == 0):
                curr_cheap_supp_avg_amt=0.0
            else:
                curr_cheap_supp_avg_amt=(current[str(month3)][procedure][category].get('Cheapest_supp_expend')/current[str(month3)][procedure][category].get('Total Expend'))*100
            
            if(curr_cheap_supp_avg_amt>past_cheap_supp_avg_amt):
                Is_CS_expend_more=1
            else:
                Is_CS_expend_more=0
                
            Result= Result.append({'Procedure':procedure,'Main_Category':category,'Is_curr_%_of_CS_expend_more':Is_CS_expend_more},ignore_index=True)
    return Result

In [58]:
output= cheapest_supplier_trend(Cheap_supp_info,month1,month2,month3)

c:\users\ganesh p umarani\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [59]:
output.to_excel('%_of_cs_result '+str(month1)+' - '+str(month2)+'.xlsx')

This is for only new month data Check if the supplier who has cheapest average price per unit for that procedure, Main category combination has got the 80% of the total spend. If not flag it , because it tells us that we are spending on costlier suppliers instead of cheap suppliers.

In [60]:
curr_month_data=data[data['Month']==month3]

In [61]:
total_expend=pd.pivot_table(curr_month_data, index = ['com_procedurename','com_bom_main_category'],values=["com_cost_price"],aggfunc={'com_cost_price': np.sum}).apply(np.ceil)

In [62]:
total_expend = pd.DataFrame(total_expend.to_records())

In [63]:
total_expend.rename(columns={'com_procedurename':'Procedure','com_bom_main_category':'Main_Category','com_cost_price':'Total_expenditure'},inplace=True)

In [64]:
Low_cost_supp=pd.pivot_table(curr_month_data, index = ['com_procedurename','com_bom_main_category'],columns='com_supplier', values=["com_cost_price","com_quantity_consumed"],aggfunc={'com_cost_price': np.mean ,'com_quantity_consumed':np.sum}).apply(np.ceil)

In [65]:
def curr_month_low_cost_supp(pt_tot_expend):
    flat_table=pd.DataFrame(pt_tot_expend.to_records())
    Result = pd.DataFrame(columns=['Procedure','Main_Category','LC_supp_name','LC_supp_cost_price','Lc_supp_qty'])
    for procedure in (flat_table['com_procedurename'].unique()).tolist():
        main_category = (flat_table.loc[flat_table['com_procedurename']== procedure]['com_bom_main_category'].unique()).tolist()
        for category in main_category:
                extracted_cost_price=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category)').dropna(axis=1).loc[:,('com_cost_price',slice(None))]
                extracted_qty=pt_tot_expend.query('(com_procedurename == @procedure) & (com_bom_main_category == @category)').dropna(axis=1).loc[:,'com_quantity_consumed']
                Cheapest_supplier_name=list(extracted_cost_price.idxmin(axis=1).values[0])[1]
                Cheapest_supp_qty=extracted_qty[Cheapest_supplier_name].values[0]
                Cheapest_cost_price=list(extracted_cost_price.min(axis=1))[0]
                Result= Result.append({'Procedure':procedure,'Main_Category':category,'LC_supp_name':Cheapest_supplier_name,
                                   'LC_supp_cost_price':Cheapest_cost_price,'Lc_supp_qty':Cheapest_supp_qty},ignore_index=True)    
    return Result        
        

In [66]:
low_cost_supp_details=curr_month_low_cost_supp(Low_cost_supp)

In [67]:
Merged=pd.merge(low_cost_supp_details,total_expend, on=['Procedure','Main_Category'])

In [68]:
Merged['%_of_tot_expen']=((Merged['LC_supp_cost_price']*Merged['Lc_supp_qty'])/Merged['Total_expenditure'])

In [69]:
Merged.to_excel('share_of_cs '+str(month1)+' - '+str(month2)+'.xlsx')

In [75]:
to_check= Merged[Merged['%_of_tot_expen']<0.80]

In [76]:
to_check.to_excel('share_of_cs_less_than_80_percent '+str(month1)+' - '+str(month2)+'.xlsx')